# Vérification du terme source de décroissance radioactive

In [ ]:
from trustutils import run  

run.introduction("Yannick Gorsse")

### Description

Le domaine d'application visé est la modélisation de l'évolution des précurseurs de neutrons retardés dans un calcul couplé avec un code de neutronique. En particulier, les précurseurs sont transportés dans les réacteurs à sels fondus. La distribution spatiale de la concentration en précurseurs (séparés en groupes associés à une constante de décroissance radioactive différente) est résolue entièrement dans TRUST via une équation de converction diffusion de la concentration : $$\partial_t c + \nabla\cdot( u c) - \nabla\cdot\left(D_\text{eff}\nabla c\right) = {\color{red}{-\lambda c}} + \text{Prod}$$

Ce terme peut être implémenté via source_constituant et champ_fonc_fonction, mais ne pourra pas être implicité, s'exposant ainsi à de potentielles instabilités.
Le terme source radioactive_decay permet d'impliciter ce terme. Pour vérifier l'implémentation on néglige les termes de convection et de diffusion : $$\partial_t c = {\color{red}{-\lambda c}} + \text{Prod}$$
La solution de cette équation est : $$c = \frac{\text{Prod}}{\lambda} (e^{-\lambda t} - 1)$$

In [ ]:
run.TRUST_parameters("1.8.0")

In [ ]:
from trustutils import run  


run.addCase("VEFPreP1B/dt0.1","decay.data")  
run.addCase("VEFPreP1B/dt1"  ,"decay.data")  
run.addCase("VEFPreP1B/dt0.1","source_constituant.data")  
run.addCase("VEFPreP1B/dt1"  ,"source_constituant.data")  

run.addCase("PolyMAC/dt0.1","decay.data")  
run.addCase("PolyMAC/dt1"  ,"decay.data")  
run.addCase("PolyMAC/dt0.1","source_constituant.data")  
run.addCase("PolyMAC/dt1"  ,"source_constituant.data")

run.addCase("PolyMAC_P0/dt0.1","decay.data")  
run.addCase("PolyMAC_P0/dt1"  ,"decay.data")  
run.addCase("PolyMAC_P0/dt0.1","source_constituant.data")  
run.addCase("PolyMAC_P0/dt1"  ,"source_constituant.data")   
run.printCases()
run.runCases()

## Comparaisons a une solution de reference 
### Comparaison

In [ ]:
from trustutils import plot
import numpy as np

a = plot.Graph("Comparaison",size=[15,8])

a.addPoint("VEFPreP1B/dt0.1/decay_C.son",label="Concentration 1 VEFPreP1B - decay - dt = 0.1 s",marker='-|')
a.addPoint("VEFPreP1B/dt1/decay_C.son",label="Concentration 1 VEFPreP1B - decay - dt = 1 s",marker='-x')

a.addPoint("PolyMAC/dt0.1/source_constituant_C.son",label="Concentration 1 VEFPreP1B - source constituant - dt = 0.1 s",marker='-*')
a.addPoint("PolyMAC/dt1/source_constituant_C.son",label="Concentration 1 VEFPreP1B - source constituant - dt = 1 s",marker='-s',markerfacecolor="None")

a.addPoint("PolyMAC/dt0.1/decay_C.son",label="Concentration 1 PolyMAC - decay - dt = 0.1 s",marker="-s")
a.addPoint("PolyMAC/dt1/decay_C.son",label="Concentration 1 PolyMAC - decay - dt = 1 s",marker='-o',markerfacecolor="None")

a.addPoint("PolyMAC/dt0.1/source_constituant_C.son",label="Concentration 1 PolyMAC - source constituant - dt = 0.1 s",marker="-o")
a.addPoint("PolyMAC/dt1/source_constituant_C.son",label="Concentration 1 PolyMAC - source constituant - dt = 1 s",marker='-^',markerfacecolor="None")

a.addPoint("PolyMAC_P0/dt0.1/decay_C.son",label="Concentration 1 PolyMAC_P0 - decay - dt = 0.1 s",marker="-^")
a.addPoint("PolyMAC_P0/dt1/decay_C.son",label="Concentration 1 PolyMAC_P0 - decay - dt = 1 s",marker='-v',markerfacecolor="None")

a.addPoint("PolyMAC_P0/dt0.1/source_constituant_C.son",label="Concentration 1 PolyMAC_P0 - source constituant - dt = 0.1 s",marker="-v")
a.addPoint("PolyMAC_P0/dt1/source_constituant_C.son",label="Concentration 1 PolyMAC_P0 - source constituant - dt = 1 s",marker='-D',markerfacecolor="None")

x = np.linspace(0,10,100)
y=-0.5/0.01*(np.exp(-0.01*x)-1) 
a.add(x,y,label="Analytique 1")   

a.label("Temps[s]","Concentration 1")
a.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=2)

a.visu()

## Comparaison

In [ ]:
from trustutils import plot
import numpy as np

a = plot.Graph("Comparaison",size=[15,8])

a.addPoint("VEFPreP1B/dt0.1/decay_C.son",compo=1,label="Concentration 2 VEFPreP1B - decay - dt = 0.1 s",marker='-|')
a.addPoint("VEFPreP1B/dt1/decay_C.son",compo=1,label="Concentration 2 VEFPreP1B - decay - dt = 1 s",marker='-x')

a.addPoint("PolyMAC/dt0.1/source_constituant_C.son",compo=1,label="Concentration 2 VEFPreP1B - source constituant - dt = 0.1 s",marker='-*')
a.addPoint("PolyMAC/dt1/source_constituant_C.son",compo=1,label="Concentration 2 VEFPreP1B - source constituant - dt = 1 s",marker='-s',markerfacecolor="None")

a.addPoint("PolyMAC/dt0.1/decay_C.son",compo=1,label="Concentration 2 PolyMAC - decay - dt = 0.1 s",marker="-s")
a.addPoint("PolyMAC/dt1/decay_C.son",compo=1,label="Concentration 2 PolyMAC - decay - dt = 1 s",marker='-o',markerfacecolor="None")

a.addPoint("PolyMAC/dt0.1/source_constituant_C.son",compo=1,label="Concentration 2 PolyMAC - source constituant - dt = 0.1 s",marker="-o")
a.addPoint("PolyMAC/dt1/source_constituant_C.son",compo=1,label="Concentration 2 PolyMAC - source constituant - dt = 1 s",marker='-^',markerfacecolor="None")

a.addPoint("PolyMAC_P0/dt0.1/decay_C.son",compo=1,label="Concentration 2 PolyMAC_P0 - decay - dt = 0.1 s",marker="-^")
a.addPoint("PolyMAC_P0/dt1/decay_C.son",compo=1,label="Concentration 2 PolyMAC_P0 - decay - dt = 1 s",marker='-v',markerfacecolor="None")

a.addPoint("PolyMAC_P0/dt0.1/source_constituant_C.son",compo=1,label="Concentration 2 PolyMAC_P0 - source constituant - dt = 0.1 s",marker="-v")
a.addPoint("PolyMAC_P0/dt1/source_constituant_C.son",compo=1,label="Concentration 2 PolyMAC_P0 - source constituant - dt = 1 s",marker='-D',markerfacecolor="None")

x = np.linspace(0,10,100)
y=-0.8/2.*(np.exp(-2.0*x)-1) 
a.add(x,y,label="Analytique 1")   

a.label("Temps[s]","Concentration 2")
a.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=2)

a.visu()